# Task 3.2: Embedding Hidden Messages Using LSB

In this task, we will be embedding a secret message into our audio file, Ex3_sound5.wav. We will then also implement an algorithm that will reverse the process of embedding.

1) Import necessary libraries

In [44]:
import wave
import numpy as np
import random

2. Make a function that will embed the secret message into the audio file

In [45]:
# self written code start

def embed_message(audio_file, output_file, secret_message, seed=42, lsb_count=1):
    # Convert message to binary
    secret_message += '\0'  # Null character to mark the end of the message
    message_bits = ''.join([f"{ord(c):08b}" for c in secret_message])

    # Open the audio file
    with wave.open(audio_file, 'rb') as wav:
        params = wav.getparams()
        frames = wav.readframes(params.nframes)
        audio_data = np.frombuffer(frames, dtype=np.int16).copy()  # Create a writable copy

    # Randomise indices
    np.random.seed(seed)
    indices = np.random.choice(len(audio_data), size=len(message_bits), replace=False)

    # Embed the message
    for i, bit in enumerate(message_bits):
        value = audio_data[indices[i]]
        for b in range(lsb_count):
            value = (value & ~(1 << b)) | (int(bit) << b)
        audio_data[indices[i]] = value

    # Save the modified audio
    with wave.open(output_file, 'wb') as wav_out:
        wav_out.setparams(params)
        wav_out.writeframes(audio_data.tobytes())

    print(f"Message embedded into {output_file}")

3. Make a function that will reverse the embedding and instead extract the secret message from the embedded audio

In [46]:
def extract_message(audio_file, seed=42, lsb_count=1):
    with wave.open(audio_file, 'rb') as wav:
        frames = wav.readframes(wav.getnframes())
        audio_data = np.frombuffer(frames, dtype=np.int16)

    np.random.seed(seed)
    indices = np.random.choice(len(audio_data), size=len(audio_data), replace=False)

    message_bits = []
    for i in indices:
        value = audio_data[i]
        for b in range(lsb_count):
            message_bits.append((value >> b) & 1)

        # Check if we have completed a character
        if len(message_bits) % 8 == 0:
            char = chr(int(''.join(map(str, message_bits[-8:])), 2))
            if char == '\0':
                break

    # Convert bits to string
    extracted_message = ''.join(
        chr(int(''.join(map(str, message_bits[i:i + 8])), 2))
        for i in range(0, len(message_bits) - 8, 8)  # Skip last null character bits
    )
    return extracted_message

4. Test the embedding function by initialising the audio file, the path and name of the file you want to save the embedded audio file into, and the secret message you want to embed

In [47]:
# Test the application
audio_file = '../exercise3/Ex3_sound5.wav'
output_file = '../exercise3/Ex3_sound5_steg.wav'
secret_message = 'An eye for an eye makes the whole world blind'

# Embed the message
embed_message(audio_file, output_file, secret_message, seed=42, lsb_count=1)

Message embedded into ../exercise3/Ex3_sound5_steg.wav


5. To test whether the embedding worked, we run the extraction function and on the output file and print out the message extracted from it

In [48]:
# Extract the message
extracted_message = extract_message(output_file, seed=42, lsb_count=1)

print(f"Extracted Message: {extracted_message}")
# self written code end


Extracted Message: An eye for an eye makes the whole world blind


For the above to work, we need to make sure that the seed and lsb_count values for both embedding and extraction match.